# EEG - Flow

## 1. Convert XDF to FIFF

The Lab Streaming Layer streams (LSL) stored in the *.xdf* file must be loaded and converted into a FIFF format.
The *.xdf* file contains:
- *eegoSports 000xxx*: the EEG data stream, with `000xxx` the serial number (S/N) of the amplifier.
- *Oddball_task*: triggers of the oddball task, duplicate of the hardware triggers from received on the `TRIGGER` channel of the EEG data stream.
- *OBS_webcam* (task-UT-only): Frame IDs from the webcam OBS video stream.
- *OBS_gameplay* (task-UT-only): Frame IDs from the screen capture OBS video stream.
- *UT_GameEvents* (task-UT-only): Events from Unreal Tournament.
- *MouseButtons* (task-UT-only): Mouse clicks and button events.
- *MousePosition* (task-UT-only): Mouse X/Y position.
- *Keyboard* (task-UT-only): Keyboard button events.

TO DO: run all 4 files at the same time in parallel? need to test on unige wired network. VPN is being way to slow

Last edit: 17.04.2023 19:16
@anguyen

In [1]:
import os

from mne import find_events

from eeg_flow.config import load_config
from eeg_flow.io import (
    add_game_events,
    add_keyboard_buttons,
    add_mouse_buttons,
    add_mouse_position,
    create_raw,
    find_streams,
    load_xdf,
)
from eeg_flow.utils.annotations import annotations_from_events
from eeg_flow.utils.bids import get_fname, get_folder, get_subfolder
from eeg_flow.utils.concurrency import lock_files

XDF_FOLDER_ROOT, DERIVATIVES_FOLDER_ROOT, _ = load_config()

The parameters of the file to process are defined below. Locks are created to prevent someone else from running the same task and from writing the same derivatives.

In [2]:
PARTICIPANT = 19        # int
GROUP       = 6         # int [1, 2, 3, 4, 5, 6, 7, 8]
TASK        = "oddball" # str [oddball, UT]
RUN         = 2         # int [1, 2]

XDF_FOLDER = get_folder(XDF_FOLDER_ROOT, PARTICIPANT, GROUP)
FNAME_STEM = get_fname(PARTICIPANT, GROUP, TASK, RUN)
#DERIVATIVES_FOLDER = get_folder(DERIVATIVES_FOLDER_ROOT, PARTICIPANT, GROUP)
#DERIVATIVES_SUBFOLDER = DERIVATIVES_FOLDER / FNAME_STEM
DERIVATIVES_SUBFOLDER = get_folder(
    DERIVATIVES_FOLDER_ROOT, PARTICIPANT, GROUP, TASK, RUN
)

# create derivatives preprocessed subfolder
os.makedirs(DERIVATIVES_SUBFOLDER, exist_ok=True)

# create locks
derivatives = [
    DERIVATIVES_SUBFOLDER / (FNAME_STEM + "_step1_oddball_annot.fif"),
    DERIVATIVES_SUBFOLDER / (FNAME_STEM + "_step1_raw.fif"),
]

if TASK == "UT":
    derivatives.append(
        DERIVATIVES_SUBFOLDER / (FNAME_STEM + "_step1_stream_annot.fif")
)

locks = lock_files(*derivatives2)

NameError: name 'derivatives2' is not defined

First, the EEG data is loaded in a [`mne.io.Raw`](https://mne.tools/stable/generated/mne.io.Raw.html) object.

In [ ]:
%%time
FNAME_XDF = XDF_FOLDER / (FNAME_STEM + "_eeg.xdf")
streams = load_xdf(FNAME_XDF)
# search stream by name and match with "eego"
eeg_stream = find_streams(streams, "eego")[0][1]
raw = create_raw(eeg_stream)

# Fix the AUX channel name/types
raw.rename_channels(
    {"AUX7": "ECG", "AUX8": "hEOG", "EOG": "vEOG", "AUX4": "EDA"}
)
raw.set_channel_types(mapping={
    "ECG": "ecg", "vEOG": "eog", "hEOG": "eog", 'EDA': 'gsr'}
)

The streams are either recording continuous information (EEG, Mouse position, UT game events), or discontinuous information (button press). A stream containing continuous information is added to the [`mne.io.Raw`](https://mne.tools/stable/generated/mne.io.Raw.html) as a `misc` channel. Note that a `misc` channel is not considered as a [data channel](https://mne.tools/stable/glossary.html#term-data-channels) by MNE.

In [ ]:
%%time
if TASK == "UT":
    # the mouse position is added on 2 channels, MouseX and MouseY.
    mouse_pos_stream = find_streams(streams, "MousePosition")[0][1]
    add_mouse_position(raw, eeg_stream, mouse_pos_stream)

    # the game events are added on 8 channels,
    # Health, Death, Primary_Assault_Ammo, Secondary_Assault_Ammo
    # Shield_Gun_Ammo, Ammo, Pick_Health_Pack, Pick_Assault_Ammo
    game_events_stream = find_streams(streams, "UT_GameEvents")[0][1]
    add_game_events(raw, eeg_stream, game_events_stream)

    # -- TODO --
    # The game events are maybe not all recording continuous data.
    # `Death`, `Pick_Health_Pack`, `Pick_Assault_Ammo` maybe?
    # If the data recorded on those channels is discontinuous
    # (event-like) then annotations would be more suited.
    #
    # The OBS streams should be added here as well.
    #
    # The "Oddball_task" stream could be added on a separate
    # synthetic stim channel, e.g. TRIGGER2 or STI101, for
    # comparison with the hardware triggers on the channel TRIGGER.

A stream containing discontinuous data (e.g. events) is added to the [`mne.io.Raw`](https://mne.tools/stable/generated/mne.io.Raw.html) as [Annotations](https://mne.tools/stable/generated/mne.Annotations.html#mne.Annotations).

In [ ]:
if TASK == "UT":
    keyboard_stream = find_streams(streams, "Keyboard")[0][1]
    add_keyboard_buttons(raw, eeg_stream, keyboard_stream)
    mouse_buttons_stream = find_streams(streams, "MouseButtons")[0][1]
    add_mouse_buttons(raw, eeg_stream, mouse_buttons_stream)

Now that all the stream information is regrouped and synchronized together, the recording can be cropped to remove the empty beginning and ending, especially if the recording on LabRecorder was not stopped immediately.

In [ ]:
events = find_events(raw, stim_channel="TRIGGER")
# get the last event that is not a button response (ie that is a stim)
for i in range(events.shape[0]-1, -1, -1):
    if events[i, 2] != 64:
        last_stim_onset = events[i, 0]
        break
tmin = max(events[0, 0] / raw.info["sfreq"] - 5, 0)
tmax = min(last_stim_onset / raw.info['sfreq'] + 5, raw.times[-1])
raw.crop(tmin, tmax)

At the time of writing, the MNE Browser (used to browse continuous recording with the `.plot()` method) does suffer from a large number of annotation. In theory, the browser performance should depend on the number of annotations displayed and not on the total number of annotations. However, this is not the case at the moment and the numerous annotations from the keyboard and the mouse buttons will make the browser very sluggish.

To avoid this issue, the non-necessary annotations are saved to disk in a separate FIFF file and are removed from the [`mne.io.Raw`](https://mne.tools/stable/generated/mne.io.Raw.html). 

*x-ref with GitHub issue: https://github.com/mne-tools/mne-qt-browser/issues/161*

In [ ]:
if TASK == "UT":
    FNAME_STREAM_ANNOT = (
        DERIVATIVES_SUBFOLDER / (FNAME_STEM + "_step1_stream_annot.fif")
    )
    raw.annotations.save(FNAME_STREAM_ANNOT, overwrite=False)
    raw.set_annotations(None)

Finally, annotations for the paradigm events are created, stored to disk in a separate FIFF file and added to the [`mne.io.Raw`](https://mne.tools/stable/generated/mne.io.Raw.html).

In [ ]:
annotations = annotations_from_events(raw, duration=0.1)
FNAME_OB_ANNOT = (
    DERIVATIVES_SUBFOLDER / (FNAME_STEM + "_step1_oddball_annot.fif")
)
annotations.save(FNAME_OB_ANNOT, overwrite=False)
raw.set_annotations(annotations)

The [`mne.io.Raw`](https://mne.tools/stable/generated/mne.io.Raw.html) is ready and can be saved before the preprocessing begins.

In [ ]:
raw.set_montage("standard_1020")

In [ ]:
%%time
FNAME_RAW = DERIVATIVES_SUBFOLDER / (FNAME_STEM + "_step1_raw.fif")
raw.save(FNAME_RAW, overwrite=False)

Regardless of the success of the task, the locks must be released.
If this step is forgotten, someone might have to remove the corresponding `.lock` file manually.

In [ ]:
for lock in locks:
    lock.release()
del locks  # delete would release anyway